In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
from gurobipy import *
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
from rsome import cpt_solver as cpt

In [2]:
# data_generation_process = "SPO_Data_Generation"
data_generation_process = "DDR_Data_Generation"

# Parameters

In [3]:
# import pyepo
# generate data
grid = (2,2) # grid size
num_train = 100 # number of training data
num_feat = 5 # size of feature
num_test = 10000
deg = 1.0 # polynomial degree
e = 1 # scale of normal std or the range of uniform. For the error term

lower = 0 # coef lower bound
upper = 1 # coef upper bound
p = num_feat # num of features
d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs
num_nodes = grid[0]*grid[0]
alpha = e # scale of normal std or the range of uniform. For the error term
mis = deg # model misspecification
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 7

In [4]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)
DataPath = os.path.dirname(grandparent_directory) + '/Data/Test0401_' + data_generation_process + "/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print("grandparent_directory:", grandparent_directory)
print("DataPath:", DataPath)
DataPath = DataPath + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_coef_seed="+str(coef_seed)+"_diff_W/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)

grandparent_directory: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Test0401_DDR_Data_Generation/


# Generate Data

In [5]:
def Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump):
# #  ****** Coef generation *********
    from Data import data_generation
    data_gen = data_generation()
    # W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, coef_seed,version = 0) 
    # print("W_star = ",W_star[0,:])
    np.random.seed(coef_seed)
    x_test_all = {}; c_test_all = {}; x_train_all = {}; c_train_all = {}; W_star_all = {}
    for iter in iteration_all:
        W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, iter,version = 0) 
        DataPath_seed = DataPath +"iter="+str(iter)+"/"
        pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)
        # #  ****** Data generation *********
        x_test_all[iter], c_test_all[iter], x_train_all[iter], c_train_all[iter], W_star_all[iter] = data_gen.generate_samples(iter,DataPath_seed,p, d, num_test, num_train, alpha, W_star, mis, num_test, 
                                data_generation_process, x_dist, e_dist, x_low, x_up, x_mean, x_var, bump) 
        # print()
    return x_test_all, c_test_all, x_train_all, c_train_all, W_star_all

# EPO(SPO+,...)

In [6]:
# from PYEPO import PyEPO_Method
# epo_runner = PyEPO_Method()
def Implement_EPO(DataPath,seed_all,batch_size,num_epochs,method_names,x_train_all,c_train_all,x_test_all,c_test_all,arcs,epo_runner):
    cost_EPO = {}
    for seed in seed_all:
        DataPath_seed = DataPath +"Seed="+str(seed)+"/"
        pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)
        # print("*** seed = ",seed,": Run EPO ******")
        cost_EPO[seed] = epo_runner.run(method_names,DataPath_seed,batch_size,num_feat,grid,num_epochs,\
                                        x_train_all[seed],c_train_all[seed],x_test_all[seed],c_test_all[seed],arcs)
    return cost_EPO

# Obtain DDR estimation

### define network

In [7]:
def _getArcs(grid):
    arcs = []
    for i in range(grid[0]):
        # edges on rows
        for j in range(grid[1] - 1):
            v = i * grid[1] + j
            arcs.append((v, v + 1))
        # edges in columns
        if i == grid[0] - 1:
            continue
        for j in range(grid[1]):
            v = i * grid[1] + j
            arcs.append((v, v + grid[1]))

    arc_index_mapping = {}
    for i in range(len(arcs)):
        arc = arcs[i]
        arc_index_mapping[arc] = i

    return arcs,arc_index_mapping

In [8]:
def solve_DDR(arcs,lamb,mu_fixed,num_nodes,x_train,c_train):
    
    N,p = x_train.shape
    N,d = c_train.shape

    # DDR
    m = Model("ddr")
    #m.setParam("DualReductions",0)
    m.setParam('OutputFlag', 0)

    W_ind = tuplelist( [(i,j) for i in range(d) for j in range(p)] )
    w0_ind = tuplelist( [i for i in range(d)])

    W_ddr = m.addVars(W_ind, lb=-GRB.INFINITY,name = "W" )
    w0_ddr = m.addVars(w0_ind, lb=-GRB.INFINITY,name = "W0" )
    alpha = m.addVars(N,num_nodes,lb=-GRB.INFINITY,name="alpha")
    expr_obj = 0
    err = []
    for n in range(N):
        cost_true_tem = c_train[n]
        expr_obj = expr_obj + alpha[n,num_nodes-1] - alpha[n,0]
        for ind in range(len(arcs)):
            cost_pred_tem = quicksum([W_ddr[ind,j] * x_train[n,j] for j in range(p)]) + w0_ddr[ind]
            err.append(cost_true_tem[ind] - cost_pred_tem)
            e = arcs[ind]
            j = e[1]
            i = e[0]
            # print("j = ",j,", i = ",i, ", e = ",e)
            m.addConstr(alpha[n,j] - alpha[n,i] >= -mu_fixed*cost_true_tem[ind] - (1-mu_fixed)*cost_pred_tem)

    m.setObjective(quicksum([err[k] * err[k] for k in range(len(err))])/N + lamb*(expr_obj)/N, GRB.MINIMIZE)
    m.optimize()
    
    W_DDR_rst = m.getAttr('x', W_ddr)
    w0_DDR_rst = m.getAttr('x', w0_ddr)
    W_ddr_val = []
    for i in range(d):
        W_ddr_val.append([W_DDR_rst[(i,j)] for j in range(p)])
    w0_ddr_val = [w0_DDR_rst[i] for i in range(d)]

    alpha_rst = m.getAttr('x', alpha)
    return w0_ddr_val,W_ddr_val,alpha_rst,m.ObjVal

# Out-of-sample performance

### Generate data

In [9]:
arcs,arc_index_mapping = _getArcs(grid)
num_arcs = len(arcs)
iteration_all = np.arange(0,10)
# obtain data
x_test_all, c_test_all, x_train_all, c_train_all, W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)

In [10]:
np.round(W_star_all[6],4)

array([[0.8929, 0.332 , 0.8212, 0.0417, 0.1077],
       [0.5951, 0.5298, 0.4188, 0.3354, 0.6225],
       [0.4381, 0.7359, 0.518 , 0.5789, 0.6454],
       [0.9902, 0.8199, 0.4132, 0.8763, 0.8238]])

### EPO performance

In [ ]:
# # EPO, including SPO, PG, LTR
# batch_size = 20
# num_epochs = 100
# method_names = ["spo+","pg","ltr"]
# method_names = ["spo+"]
# from PYEPO import PyEPO_Method
# epo_runner = PyEPO_Method()

# cost_EPO_all = Implement_EPO(DataPath,[1],batch_size,num_epochs,method_names,x_train_all,c_train_all,x_test_all,c_test_all,arcs,epo_runner)
# print("SPO+Cost = ",np.nanmean(cost_EPO_all[1]["SPO"]))

## Oracle performance

In [15]:
from Peformance import performance_evaluation
perfs = performance_evaluation()
cost_Oracle_all = {}; cost_Oracle_avg = np.zeros(len(iteration_all))
for iter in iteration_all:
    cost_Oracle_all[iter] = perfs.compute_Cost_with_Prediction(arcs,np.ones(num_arcs)*bump, W_star_all[iter],grid,c_test_all[iter],x_test_all[iter])
    cost_Oracle_avg[iter] = np.nanmean(cost_Oracle_all[iter])
print("Oracle Cost = ",np.round(cost_Oracle_avg,4))

Oracle Cost =  [13.3035 13.3218 13.6684 13.2319 12.9373 13.5863 13.4639 13.3154 13.3031
 13.5261]


### OLS performance

In [16]:
from OLS import ols_method
ols_method_obj = ols_method()
W_ols_all = {}; w0_ols_all = {}; t_ols_all = {}; obj_ols_all = {}
cost_OLS_all = {};cost_OLS_avg = np.zeros(len(iteration_all))
for iter in iteration_all:
    # compute OLS performance
    W_ols_all[iter], w0_ols_all[iter], t_ols_all[iter], obj_ols_all[iter] = ols_method_obj.ols_solver("",x_train_all[iter], c_train_all[iter])
    cost_OLS_all[iter] = perfs.compute_Cost_with_Prediction(arcs,w0_ols_all[iter],W_ols_all[iter], grid,c_test_all[iter],x_test_all[iter])
    cost_OLS_avg[iter] = np.nanmean(cost_OLS_all[iter])
print("OLS cost = ",np.round(cost_OLS_avg,4))

OLS cost =  [13.3374 13.3344 13.6979 13.24   12.9433 13.5946 13.4978 13.3584 13.3254
 13.5729]


### DDR performance

In [18]:
def obtain_DDR_out_of_sample_performance(mu_all,lamb_all,num_nodes,x_train,c_train,x_test,c_test,perfs):
    lamb_index = 0
    cost_DDR = {}; w0_ddr_dict = {}; W_ddr_dict = {}
    cost_DDR_avg = np.zeros((len(mu_all),len(lamb_all)))
    mu_index = 0
    for mu in mu_all:
        lamb_index = 0
        for lamb in lamb_all:
            w0_ddr_dict[mu,lamb],W_ddr_dict[mu,lamb],alpha_rst,obj_ddr = solve_DDR(arcs,lamb,mu,num_nodes,x_train,c_train)
            cost_DDR[mu,lamb] = perfs.compute_Cost_with_Prediction(arcs,w0_ddr_dict[mu,lamb],W_ddr_dict[mu,lamb], grid,c_test,x_test)
            cost_DDR_avg[mu_index,lamb_index] = np.nanmean(cost_DDR[mu,lamb])
            lamb_index = lamb_index + 1
        # print("cost_DDR_avg=",np.round(cost_DDR_avg[0,:],4))
        mu_index = mu_index + 1
    return cost_DDR,w0_ddr_dict,W_ddr_dict,cost_DDR_avg

In [19]:
mu_all = np.arange(0.01,1.0,0.1)
lamb_all = np.round(np.arange(0.01,1,0.1),4)
regret_all = {}; cost_DDR_all = {}; w0_ddr_all = {}; W_ddr_all = {}
cost_DDR_avg_all = np.zeros((len(mu_all),len(lamb_all)))
regred_DDR_vs_OLS_all = np.zeros((len(mu_all),len(lamb_all)))

for iter in iteration_all:
    cost_DDR_all[iter],w0_ddr_all[iter],W_ddr_all[iter],cost_DDR_avg = obtain_DDR_out_of_sample_performance(mu_all,lamb_all,num_nodes,x_train_all[iter],c_train_all[iter],x_test_all[iter],c_test_all[iter],perfs)
    cost_DDR_avg_all = cost_DDR_avg_all + cost_DDR_avg
    regert_tem = (np.nanmean(cost_OLS_all[iter]) - cost_DDR_avg)/(np.nanmean(cost_OLS_all[iter]) - np.nanmean(cost_Oracle_all[iter])) * 100
    print("iter=",iter,",DDR_vs_OLS = ",np.round(regert_tem,2))
    regred_DDR_vs_OLS_all = regred_DDR_vs_OLS_all + regert_tem
cost_DDR_avg_all = cost_DDR_avg_all/len(iteration_all)
regred_DDR_vs_OLS_all = regred_DDR_vs_OLS_all/len(iteration_all)
# print("regred_DDR_vs_OLS_all=",regred_DDR_vs_OLS_all)

iter= 0 ,DDR_vs_OLS =  [[-1.900e+00 -2.740e+00 -3.170e+00 -8.600e-01  2.600e+00  2.800e-01
   2.720e+00  2.650e+00  2.920e+00  3.940e+00]
 [-5.900e-01 -2.920e+00 -4.490e+00 -5.690e+00 -1.254e+01 -1.225e+01
  -1.379e+01 -1.525e+01 -1.894e+01 -2.331e+01]
 [-2.700e-01  1.000e-02  2.420e+00  4.670e+00 -9.100e+00 -2.600e+00
  -6.450e+00 -1.251e+01 -2.371e+01 -1.619e+01]
 [ 1.300e-01  5.300e-01  2.100e-01 -9.000e-02  2.800e-01  3.000e-02
  -6.330e+00 -1.570e+00  8.680e+00  2.850e+00]
 [ 0.000e+00 -1.400e-01  2.640e+00  1.230e+00  2.250e+00  3.900e-01
   2.600e-01  4.460e+00  3.690e+00  8.200e-01]
 [ 0.000e+00 -7.400e-01  2.320e+00  4.450e+00  2.690e+00  2.520e+00
   8.100e-01  2.830e+00  5.000e-02  1.400e-01]
 [ 0.000e+00 -1.150e+00 -1.090e+00 -2.500e-01 -3.800e-01  1.110e+00
   1.870e+00 -2.070e+00 -2.160e+00 -4.600e+00]
 [ 4.100e-01 -2.840e+00 -1.590e+00  2.000e-01 -5.800e-01 -4.300e-01
  -1.480e+00 -3.850e+00 -3.910e+00 -3.480e+00]
 [ 0.000e+00 -3.080e+00 -2.800e+00 -1.560e+00 -1.120e+00 

In [21]:
print("regred_DDR_vs_OLS_all=",np.round(regred_DDR_vs_OLS_all,2))

regred_DDR_vs_OLS_all= [[-0.54  0.39  1.51  0.04 -1.65 -2.48 -2.03 -3.17 -5.63 -8.23]
 [-0.08  0.43 -3.42 -0.12 -0.62  0.64 -0.97  1.75  3.02 -1.49]
 [-0.38 -0.77 -0.46  4.85  3.46  6.26  2.49  8.02  3.16 -1.91]
 [-0.13  0.04 -0.79  0.45  2.    1.34  0.54 -2.46  0.39 -1.92]
 [-0.38  0.78  3.57 -0.3   1.29 -1.15 -3.03 -4.24 -9.78 -7.07]
 [-0.72 -1.09  1.76  1.21  0.46  1.98  0.88 -2.41 -0.53 -0.11]
 [ 0.09 -2.75 -2.88 -1.58 -1.43 -5.86 -5.44 -5.71 -4.79 -3.27]
 [ 0.06 -0.89 -0.36  0.89 -0.44 -0.86 -2.57 -3.72 -6.13 -5.55]
 [-0.06  0.03  1.07  1.54  1.58  1.65 -0.44  0.51  0.37 -1.33]
 [-0.06 -0.19 -0.08 -0.1  -1.02  0.23  0.39  0.72  0.19 -1.65]]


In [ ]:
# with open(DataPath+'cost_OLS_all.pkl', "wb") as tf:
#     pickle.dump(cost_OLS_all,tf)
# with open(DataPath+'cost_Oracle_all.pkl', "wb") as tf:
#     pickle.dump(cost_Oracle_all,tf)
# with open(DataPath+'cost_DDR_all.pkl', "wb") as tf:
#     pickle.dump(cost_DDR_all,tf)

In [ ]:
# seed = 1
# lamb = 1
# cost_OLS_esti = x_test_all[seed] @ np.asarray(W_ols_all[seed]).T + w0_ols_all[seed]
# cost_DDR_esti = x_test_all[seed] @ np.asarray(W_ddr_all[seed,mu][lamb]).T + w0_ddr_all[seed,mu][lamb]

# S_test_index = 3
# print("cost_DDR_esti=",np.round(cost_DDR_esti[S_test_index,],2))
# # print("cost:(0,1)-->(1,3) = ",cost_DDR_esti[S_test_index,0] + cost_DDR_esti[S_test_index,2])
# # print("cost:(0,2)-->(2,3) = ",cost_DDR_esti[S_test_index,1] + cost_DDR_esti[S_test_index,3])

# print("cost_OLS_esti=",np.round(cost_OLS_esti[S_test_index,],2))
# # print("cost:(0,1)-->(1,3) = ",cost_OLS_esti[S_test_index,0] + cost_OLS_esti[S_test_index,2])
# # print("cost:(0,2)-->(2,3) = ",cost_OLS_esti[S_test_index,1] + cost_OLS_esti[S_test_index,3])


# print("cost_Oracle=",np.round(c_test_all[seed][S_test_index],2))
# # print("cost:(0,1)-->(1,3) = ",c_test_all[seed][S_test_index,0] + c_test_all[seed][S_test_index,2])
# # print("cost:(0,2)-->(2,3) = ",c_test_all[seed][S_test_index,1] + c_test_all[seed][S_test_index,3])

# Plot figures

In [ ]:
def cross_compare2plus(c_item, c_base, c_oracle):
    c_item = np.asarray(c_item)
    c_base = np.asarray(c_base)
    c_oracle = np.asarray(c_oracle)

    N = len(c_item)
    c_diff = c_item - c_base
    lbel = np.zeros((N,1))
    
    equals = np.sum(c_diff == 0)
    wins = np.sum(c_diff < 0)
    lose = np.sum(c_diff > 0)
    
    lbel[c_diff < 0] = 1
    lbel[c_diff > 0] = -1
    
    # print("Num_train =",N,",Num_equals =",equals,",Num_wins =",wins,",Num_lose =",lose)
    # print("base cost = ", np.mean(c_base),",item cost = ",np.mean(c_item))
    if N == equals:
        win_ratio = 0.5
    else:
        win_ratio = wins/(N - equals)
    # cost_reduction = (np.mean(c_base) - np.mean(c_item) )/np.abs(np.mean(c_oracle))
    regret_reduction = (np.nanmean(c_base) - np.nanmean(c_item))/np.abs(np.nanmean(c_base) - np.nanmean(c_oracle))
    return lbel, win_ratio, regret_reduction

### DDR figure

In [ ]:
# H2H_DDR_vs_OLS_all = {}; regret_reduction_DDR_vs_OLS_all = {}
# for mu in mu_all:
#     for lamb in lamb_all:
#         H2H_DDR_vs_OLS_arr = np.zeros(len(iteration_all)); regret_reduction_DDR_vs_OLS_arr = np.zeros(len(iteration_all))
#         # print("lamb = ",lamb)
#         for iter in iteration_all:
#             lbel, H2H_DDR_vs_OLS_arr[iter-1], regret_reduction_DDR_vs_OLS_arr[iter-1] = cross_compare2plus(cost_DDR_all[iter][mu,lamb],cost_OLS_all[iter], cost_Oracle_all[iter])
#         H2H_DDR_vs_OLS_all[mu,lamb] = H2H_DDR_vs_OLS_arr; regret_reduction_DDR_vs_OLS_all[mu,lamb] = regret_reduction_DDR_vs_OLS_arr

In [ ]:
# mu = mu_all[0]
# lamb = lamb_all[7]
# import Figures
# file_name = DataPath + "figure_mu="+str(mu)+"_lamb="+str(lamb)
# Figures.figure_plot_upleft(H2H_DDR_vs_OLS_all[mu,lamb]*100, regret_reduction_DDR_vs_OLS_all[mu,lamb], file_name, size = (5, 5), move = [-0.12, 0.04, 0.35, 0.55], 
#                     ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

In [ ]:
# import matplotlib.pyplot as plt
# fig = plt.figure()
# ax = plt.subplot()
# ax.plot(lamb_all,regret_all[seed,mu])

### SPO figure

In [ ]:
# H2H_SPO_vs_OLS_arr = np.zeros(len(seed_all)); regret_reduction_SPO_vs_OLS_arr = np.zeros(len(seed_all))
# seed_index = 0
# for seed in seed_all:
#     lbel, H2H_SPO_vs_OLS_arr[seed_index], regret_reduction_SPO_vs_OLS_arr[seed_index] = cross_compare2plus(cost_EPO_all[seed]["SPO"],cost_OLS_all[seed], cost_Oracle_all[seed])
#     seed_index = seed_index + 1
# H2H_SPO_vs_OLS_all = H2H_SPO_vs_OLS_arr; regret_reduction_SPO_vs_OLS_all = regret_reduction_SPO_vs_OLS_arr

In [ ]:
# Figures.figure_plot_upleft(H2H_SPO_vs_OLS_all*100, regret_reduction_SPO_vs_OLS_all, "", size = (5, 5), move = [-0.12, 0.04, 0.35, 0.55], 
#                     ysame = 0, yrange = [6,6], sublabel = '', ypio = 0)

In [ ]:
# regret_reduction_SPO_vs_OLS_arr